# Introduction

This project will focus on exploring the capabilities of Bayesian optimization, specifically employing BayBE, in the discovery of novel corrosion inhibitors for materials design. Initially, we will work with a randomly chosen subset from a comprehensive database of electrochemical responses of small organic molecules. Our goal is to assess how Bayesian optimization can speed up the screening process across the design space to identify promising compounds. We will compare different strategies for incorporating alloy information, while optimizing the experimental parameters with respect to the inhibitive performance of the screened compounds.

# Initizalization

Loading libraries and data files:

In [1]:
import pandas as pd
import numpy as np
from baybe import Campaign

df_AA2024 = pd.read_excel('data/filtered_AA2024.xlsx')
df_AA1000 = pd.read_excel('data/filtered_AA1000.xlsx')
df_Al = pd.read_excel('data/filtered_Al.xlsx')

/home/vscode/.local/lib/python3.10/site-packages/baybe/telemetry.py:222: UserWarning: WARNING: BayBE Telemetry endpoint https://public.telemetry.baybe.p.uptimize.merckgroup.com:4317 cannot be reached. Disabling telemetry. The exception encountered was: ConnectionError, HTTPConnectionPool(host='verkehrsnachrichten.merck.de', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7f7ebef1cac0>: Failed to resolve 'verkehrsnachrichten.merck.de' ([Errno -2] Name or service not known)"))
  warnings.warn(
/home/vscode/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_AA1000.describe()

,Time_h,pH,Inhib_Concentrat_M,Salt_Concentrat_M,Efficiency
count,1966.000000,1966.000000,1.966000e+03,1966.000000,1966.000000
mean,71.332309,3.626585,2.918525e-02,0.077655,48.383540
std,154.594325,4.089542,2.442926e-01,0.217774,163.164194
min,0.000000,-0.600000,1.000000e-07,0.000000,-4834.000000
25%,1.670000,0.000000,1.000000e-04,0.000000,35.602500
50%,24.000000,1.000000,1.000000e-03,0.000000,61.490000
75%,24.000000,7.000000,2.000000e-03,0.100000,82.000000
max,720.000000,14.301030,3.280000e+00,2.000000,100.000000


In [3]:
df_AA2024.describe()

,Time_h,pH,Inhib_Concentrat_M,Salt_Concentrat_M,Efficiency
count,611.000000,611.000000,611.000000,611.000000,611.000000
mean,135.801964,6.342062,0.006808,0.145450,26.736841
std,201.683867,2.529080,0.014059,0.200575,288.788317
min,0.500000,0.000000,0.000010,0.000000,-4834.000000
25%,24.000000,4.000000,0.000500,0.010000,30.000000
50%,24.000000,7.000000,0.001000,0.100000,58.000000
75%,144.000000,7.000000,0.003000,0.100000,87.950000
max,672.000000,10.000000,0.100000,0.600000,100.000000


In [4]:
df_Al.describe()

,Time_h,pH,Inhib_Concentrat_M,Salt_Concentrat_M,Efficiency
count,1966.000000,1966.000000,1.966000e+03,1966.000000,1966.000000
mean,71.332309,3.626585,2.918525e-02,0.077655,48.383540
std,154.594325,4.089542,2.442926e-01,0.217774,163.164194
min,0.000000,-0.600000,1.000000e-07,0.000000,-4834.000000
25%,1.670000,0.000000,1.000000e-04,0.000000,35.602500
50%,24.000000,1.000000,1.000000e-03,0.000000,61.490000
75%,24.000000,7.000000,2.000000e-03,0.100000,82.000000
max,720.000000,14.301030,3.280000e+00,2.000000,100.000000


# Data Processing

In [8]:
df = df_AA2024

# Data Anaylsis

# Bayesian Optimization

## Search Space

Define parameters

In [23]:
from baybe.parameters import NumericalDiscreteParameter, NumericalContinuousParameter
from baybe.searchspace import SearchSpace

parameters = [
NumericalContinuousParameter(
    name="Time (h)",
    bounds=(df['Time_h'].min(), df['Time_h'].max()),
),
NumericalContinuousParameter(
    name="pH",
    bounds=(1, 14),
    ),  
NumericalContinuousParameter(
    name="Inhibitor Concentration (M)",
    bounds=(df['Inhib_Concentrat_M'].min(), df['Inhib_Concentrat_M'].max()),
    ),
NumericalContinuousParameter(
    name="Salt_Concentrat_M",
    bounds=(df['Salt_Concentrat_M'].min(), df['Salt_Concentrat_M'].max()),
    )
]

TypeError: NumericalContinuousParameter.__init__() got an unexpected keyword argument 'tolerance'

## Objective

In [19]:
from baybe.targets import NumericalTarget
from baybe.objective import Objective

target = NumericalTarget(
    name="Efficiency",
    mode="MAX",
)
objective = Objective(mode="SINGLE", targets=[target])


In [20]:
from baybe.searchspace import SearchSpace

searchspace = SearchSpace.from_product(parameters)

## Recommender

In [21]:
from baybe import Campaign

campaign = Campaign(searchspace, objective)

df_recommend = campaign.recommend(batch_size=3)
print(df_recommend)

     Time (h)        pH  Inhibitor Concentration (M)  Salt_Concentrat_M
0  139.744749  9.907613                     0.003735           0.418879
1  516.446510  8.229525                     0.009470           0.463482
2   99.505625  7.063403                     0.030336           0.543470


ValueError: operands could not be broadcast together with shapes (6,) (3,4) 

# Benchmarking

# Transfer Learning